In [19]:
import os
import xarray as xr
import rioxarray
from netCDF4 import Dataset

# Meteo files

In [2]:
data_root = os.path.expanduser("~/mnt/Data-Raw-RE/27_Natural_Resources-RE/99_Meteo_Public/MeteoSwiss_netCDF/__griddedData/lv95updated")
data_files = [f for f in os.listdir(data_root) if f.endswith(".nc")]

In [3]:
data_files[2]

'topo.ch_ch01h.swiss.lv95.nc'

In [36]:
data_files[1]

'RhiresD_ch01h.swiss.lv95_200101010000_200112310000.nc'

In [4]:
ds_gridh = xr.open_dataset(os.path.join(data_root, data_files[2]), decode_times=False)

In [37]:
ds_old = xr.open_dataset(os.path.join(data_root, data_files[1]))

In [34]:
ds_gridh

<xarray.Dataset> Size: 1MB
Dimensions:                 (E: 370, N: 265, time: 1)
Coordinates:
  * E                       (E) float64 3kB 2.474e+06 2.476e+06 ... 2.844e+06
  * N                       (N) float64 2kB 1.06e+06 1.06e+06 ... 1.324e+06
  * time                    (time) float64 8B 100.0
    lon                     (N, E) float32 392kB ...
    lat                     (N, E) float32 392kB ...
Data variables:
    swiss_lv95_coordinates  float32 4B ...
    height                  (time, N, E) float32 392kB ...
Attributes:
    Conventions:  CF-1.6
    institution:  Federal Office of Meteorology and Climatology MeteoSwiss
    References:

In [54]:
ds.RhiresD.isnull().sum()

<xarray.DataArray 'RhiresD' ()> Size: 8B
array(12760590)
Coordinates:
    swiss_lv95_coordinates  int64 8B 0

## Reproject

In [45]:
ds = ds.rename({"E": "x", "N": "y"})
ds = ds.drop_vars(["lat", "lon"])

In [47]:
ds.rio.write_crs("EPSG:2056", inplace=True)
ds.rio.reproject("EPSG:32632")

/home/f80873755@agsad.admin.ch/virtualenvs/10_cropcoveo/lib/python3.11/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


<xarray.Dataset> Size: 149MB
Dimensions:                 (x: 375, y: 272, time: 366)
Coordinates:
  * x                       (x) float64 3kB 2.53e+05 2.54e+05 ... 6.269e+05
  * y                       (y) float64 2kB 5.327e+06 5.326e+06 ... 5.056e+06
  * time                    (time) datetime64[ns] 3kB 2016-01-01 ... 2016-12-31
    swiss_lv95_coordinates  int64 8B 0
Data variables:
    RhiresD                 (time, y, x) float32 149MB nan nan nan ... nan nan
Attributes:
    Conventions:  CF-1.8
    institution:  Data produced at Federal Office of Meteorology and Climatol...
    title:        Spatial Climate Analyses Dataset of MeteoSwiss (RhiresD)
    history:      2023-11-17 15:03:56 - generated by gridmch 7.28.1
    References:   Information on the data is available at \nhttps://www.meteo...

## Regrid

In [74]:
def lon_lat_grid(self):
        nx, ny = self.xy_shape
        lon_left, lat_bottom, lon_right, lat_top = self.bbox

        lon_grid = np.linspace(lon_left, lon_right-self.resolution, nx)
        lat_grid = np.linspace(lat_top, lat_bottom+self.resolution, ny)

        return lon_grid, lat_grid


def regrid_product_cube(product_cube, lon_lat_grid):

    # Store original data types
    original_dtypes = {var: product_cube[var].dtype for var in product_cube.variables}

    if ("x" in product_cube.coords) and ("y" in product_cube.coords):

        x, y = product_cube.x.values, product_cube.y.values
        lon_grid, lat_grid = lon_lat_grid
        
        product_cube = product_cube.interp(x = lon_grid, y = lat_grid, method = "nearest")
        
        #product_cube["x"], product_cube["y"] = lon_grid, lat_grid

        product_cube = product_cube.rename({"x": "lon", "y": "lat"})

    """ 
    # Restore original data types with rounding
    for var, dtype in original_dtypes.items():
        if var == 'x':
            var = 'lon'
        if var == 'y':
            var = 'lat'
        if np.issubdtype(dtype, np.integer):
            product_cube[var] = product_cube[var].round().astype(dtype)
        else:
            product_cube[var] = product_cube[var].astype(dtype)

        
    product_cube.attrs = {}
    """

    return product_cube

In [75]:
import numpy as np

lon = ds.x.values-50
lat = ds.y.values-50

In [76]:
regrid = regrid_product_cube(ds, [lon, lat])

In [79]:
regrid.lon

<xarray.DataArray 'lon' (lon: 370)> Size: 3kB
array([2474450., 2475450., 2476450., ..., 2841450., 2842450., 2843450.])
Coordinates:
    swiss_lv95_coordinates  int64 8B 0
  * lon                     (lon) float64 3kB 2.474e+06 2.475e+06 ... 2.843e+06
Attributes:
    units:          meters
    long_name:      swiss easting (lv95)
    standard_name:  projection_x_coordinate

In [80]:
ds.x

<xarray.DataArray 'x' (x: 370)> Size: 3kB
array([2474500., 2475500., 2476500., ..., 2841500., 2842500., 2843500.])
Coordinates:
    swiss_lv95_coordinates  int64 8B 0
  * x                       (x) float64 3kB 2.474e+06 2.476e+06 ... 2.844e+06
Attributes:
    units:          meters
    long_name:      swiss easting (lv95)
    standard_name:  projection_x_coordinate

# Integrate with custom grid

In [84]:
import geopandas as gpd

In [85]:
grid_path = '~/mnt/eo-nas1/eoa-share/projects/012_EO_dataInfrastructure/Project layers/gridface_s2tiles_CH.shp'
grid = gpd.read_file(grid_path)

In [91]:
# Loop through meteofiles, extract necessary info (time resolution, variable)

for i, row in grid.iterrows(): 
  patch = row.geometry
  minx, miny, maxx, maxy = patch.bounds
  lon_lat_grid = [np.arange(minx, maxx, 10), np.arange(miny, maxy, 10)]

  regrid = regrid_product_cube(ds, lon_lat_grid)

  # Add metadata
  # Save the data to zarr meteo_var_minx_maxy_resolution.zarr
  break